<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/floodplain_and_terrace_examples/get_terraces_command_line_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terrace extraction, command line version

Last updated by Simon M Mudd 10-Feb-2024

This notebook does an example terrace extraction using only the command line. It works on google colab, but will also work in any linux environment where lsdtopotools is installed. For example, on the University of Edinburgh's school of GeoSciences servers.

In colab (where this was written) you use the `!` sybmbol to call the command line. If you are working in a command line, then you don't need the `!` symbol.

For example, if there is the command here of
```
!tar -xzf rasters_USGS10m.tar.gz
```

then in your command line (which will have a `$` prompt, or in Docker a `#` prompt) you would just write:

```
$ tar -xzf rasters_USGS10m.tar.gz
```



## Stuff we need to do if you are in colab (not required in on school of geosciences servers)

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [ ]:
!lsdtt-basic-metrics -v

Now we install `lsdviztools`. We need to test if this is version 0.4.12:

In [ ]:
!pip install lsdviztools  &> /dev/null
import lsdviztools
lsdviztools.__version__

We also need the gdal command line tools if we are working in colab:

In [ ]:
!apt install gdal-bin  &> /dev/null

## Get the raw data

1. Go onto the opentopography website for 10m USGS data (https://portal.opentopography.org/raster?opentopoID=OTNED.012021.4269.1) and select a region.
2. Select geotiff as the format.
3. Download.
4. Your data will come as a file with the extension `.tar.gz`
5. Copy this file into google colab or your file system. For google colab, click to the left of this screen where you see the little folder symbol. You can drag and drop the file.
6. We can make sure the file is in the google colab system by calling the list command (`ls`), in google colab to call the underlying operating system you use the `!` character:

In [ ]:
!ls *.tar.gz

## Process the data into the correct format

A `.tar.gz` file is a tye of zipped file (the `tar` is for tarball and `gz` is for gzip). You can remove these two layers of zipping with one command to a linux program called `tar`. Again, to access the underlying system on google colab use the `!` command.

You need to use some flags (`-xzf`) which I will not explain other than they do the bits you need to get the file you need.

In [ ]:
!tar -xzf rasters_USGS10m.tar.gz

Now we can see what we have (I will tell you we will now have a new `tif` file:

In [ ]:
!ls -l

Now we convert the file using a gdal command line program `gdalwarp`.

You will need to modify the line below with

1. Your UTM zone. Look it up on this image: https://www.dmap.co.uk/utmworld.htm
2. The name of your input file (the default for opentopography 10m data is `output_USGS10m.tif`
3. The name you want your data to be (need the `bil` extension, I am calling it `my_DEM.bil`
4. The grid resolution after the flag `-tr` (here 10 10 since the cells are 10x10m).
5. The resampling method `-r` (you can use `cubic` or `bilinear`...the default is `nearest` which will create a DEM that looks like it is corrugated)

In [ ]:
!gdalwarp -t_srs '+proj=utm +zone=10 +north +datum=WGS84' -of ENVI -r cubic -tr 10 10 output_USGS10m.tif my_DEM.bil

Note, the DEM used in this example has its lower left corner around here:

`36.18557118689764, -121.52769549262867`

and the upper right corner around here:

`36.32311785135548, -121.29145457790464`

## First command line call: a basic hillshade

All the `lsdtopotools` command line tools need a set of instructions. We give the instructions using little text files, that we call driver files.

It is a bit tedious to write these in a python environment, if you are doing this in a Linux terminal you should use a text editor to write the files.

Here is a very basic example where we `remove_seas` (this turns any pixel with an elevation <= 0 to nodata) and `write_hillshade` (which writes a hillshade raster):

In [ ]:
# Open the file in write mode ('w')
with open('run_01.driver', 'w') as f:
    # Write multiple lines to the file
    f.write("read fname: my_DEM\n")
    f.write("write fname: my_DEM\n")
    f.write("remove_seas: true\n")
    f.write("write_hillshade: true\n")


!cat run_01.driver

Now that we have a driver file, we can run it using `lsdtt-basic-metrics`, which has many of the most common topographic analysis routines:

In [ ]:
!lsdtt-basic-metrics run_01.driver

## Basic plotting with lsdviztools

`lsdtopotools` outputs are all readable by your favourite GIS. But we can also make simple images using some plotting routines made available by installing `lsdviztools`.

In [ ]:
!lsdtt_plotbasicrasters -dir ./ -fname my_DEM -PD True

You get a `.png` image in a subdirectory `raster_plot`, which you can just look at. In python you can also display it here.

In [ ]:
basins_img = "./raster_plots/my_DEM_drape.png"
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

## Getting channel sources

To run the terrace routine we need to get channel sources (the pixels where channels start). So lets write another driver file

In [ ]:
# Open the file in write mode ('w')
with open('run_02.driver', 'w') as f:
    # Write multiple lines to the file
    f.write("read fname: my_DEM\n")
    f.write("write fname: my_DEM\n")
    f.write("remove_seas: true\n")
    f.write("print_wiener_channels: true\n")
    f.write("print_sources_to_csv: true\n")


!cat run_02.driver

In [ ]:
!lsdtt-channel-extraction run_02.driver

This makes a few `csv` files that have the channel head locations, we can check on them with the `ls` command.

In [ ]:
!ls *.csv

### Look at sources using folium

If you are in a python environment, you can look at the sources using `folium`. If you are just in a command line environment, you can load the points in a GIS.

In [ ]:
import pandas as pd

sources_AT_df = pd.read_csv("my_DEM_ATsources.csv")
sources_W_df = pd.read_csv("my_DEM_Wsources.csv")

In [ ]:
# This is for the wiener points

import folium

#create a map
this_map = folium.Map(prefer_canvas=True, zoom_start=13)

esri = folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
    control=True
).add_to(this_map)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
sources_AT_df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

## Get the floodplains

Now we can get the floodplains. We need to write another driver file

In [ ]:
# Open the file in write mode ('w')
with open('run_03.driver', 'w') as f:
    # Write multiple lines to the file
    f.write("read fname: my_DEM\n")
    f.write("write fname: my_DEM\n")
    f.write("remove_seas: true\n")
    f.write("CHeads_file: my_DEM_ATsources.csv\n")
    f.write("fill_floodplain: false\n")


!cat run_03.driver

And now run the floodplain extraction. When I tested this, for some unkown reason it needed to be run twice.

In [ ]:
!lsdtt-valley-metrics run_03.driver

In [ ]:
!ls *valley*

In [ ]:
!lsdtt_plotbasicrasters -dir ./ -fname my_DEM -PD True -drape_fname my_DEM_valley -drape_cbar_label "valley pixels" -out_fname valley_drape

## Get a channel for terrace extraction and run it

In the terrace code you can choose the channel along which you want to extract terraces. You need to give the start and end point, in latitude and longitude.

This is ingested by the code as a csv, which you can write in a text editor. But I'm in python so I will write the file here:

In [ ]:
# Open the file in write mode ('w')
with open('channel_start_end.csv', 'w') as f:
    # Write multiple lines to the file
    f.write("latitude,longitude\n")
    f.write("36.23580231079953,-121.48777139483659\n")
    f.write("36.28345309047785,-121.31396859423032\n")


!cat channel_start_end.csv

Now we make the driver file that uses this channel to get the terraces:

In [ ]:
# Open the file in write mode ('w')
with open('run_04.driver', 'w') as f:
    # Write multiple lines to the file
    f.write("read fname: my_DEM\n")
    f.write("write fname: my_DEM\n")
    f.write("remove_seas: true\n")
    f.write("CHeads_file: my_DEM_ATsources.csv\n")
    f.write("fill_floodplain: false\n")
    f.write("extract_single_channel: true\n")
    f.write("channel_source_fname: channel_start_end.csv\n")
    f.write("get_terraces: true\n")


!cat run_04.driver

Run the code (this will take a little while, it is quite computationally expensive):

In [ ]:
!lsdtt-valley-metrics run_04.driver

Again, you can plot the data by loading it into a GIS, or use one of the `lsdviztools` scripts.

In [ ]:
!lsdtt_plotbasicrasters -dir ./ -fname my_DEM -PD True -drape_fname my_DEM_terraces -drape_cbar_label "terrace pixels" -out_fname terrace_drape